## Importing Libraries

In [52]:
import pandas as pd
import numpy as np
from datetime import datetime
import ast

import industryMap

## Importing data

In [53]:
# Importing File
user_df = pd.read_csv('../SavedData/Lloyds_users.csv')

# Changing the lists back to normal columns
user_df['user_biz_interaction'] = user_df['user_biz_interaction'].apply(ast.literal_eval)
user_df['user_biz_interaction_frequency'] = user_df['user_biz_interaction_frequency'].apply(ast.literal_eval)
user_df['user_biz_spend'] = user_df['user_biz_spend'].apply(ast.literal_eval)
user_df['user_user_interaction'] = user_df['user_user_interaction'].apply(ast.literal_eval)
user_df['user_user_interaction_frequency'] = user_df['user_user_interaction_frequency'].apply(ast.literal_eval)
user_df['user_user_spend'] = user_df['user_user_spend'].apply(ast.literal_eval)
user_df['user_industry_interaction'] = user_df['user_industry_interaction'].apply(ast.literal_eval)
user_df['user_industry_interaction_frequency'] = user_df['user_industry_interaction_frequency'].apply(ast.literal_eval)
user_df['user_industry_spend'] = user_df['user_industry_spend'].apply(ast.literal_eval)
user_df['user_date_transaction'] = user_df['user_date_transaction'].apply(ast.literal_eval)
user_df['user_date_transaction_freq'] = user_df['user_date_transaction_freq'].apply(ast.literal_eval)

## Get Relative Number of Users

In [74]:
user_num = len(user_df['user_id'])

# If you pick a threshold to high, some industries won't have enough unique users to build the dictionary effectively
# Anywhere above 4% starts to include anyone who has spent at an Accessory store, and in fact runs out of people to include in the list
# This causes the dictionary entries to be of different lengths
user_perc = 3
user_perc_threshold = int(round(user_num * (user_perc / 100), 0))

## Industry Big Spenders

In [75]:
industry_list = list(set(industryMap.industry_categories.values()))

industry_user_spend = {}

for industry in industry_list:
    
    user_spend_dict = {}
    top_n = user_perc_threshold
    
    for row in range(0, len(user_df)):
        if(industry in user_df['user_industry_interaction'][row]):
            industry_index = user_df['user_industry_interaction'][row].index(industry)
            industry_spend = user_df['user_industry_spend'][row][industry_index]
            
            user_spend_dict[user_df['user_id'][row]] = industry_spend
    
    sorted_user_spend_dict = dict(sorted(user_spend_dict.items(), key=lambda item: item[1], reverse=True)[:top_n])
            
    sorted_user_spend_user_list = list(sorted_user_spend_dict.keys())
    sorted_user_spend_spend_list = list(sorted_user_spend_dict.values())
            
    industry_user_spend[industry + ' users'] = sorted_user_spend_user_list
    industry_user_spend[industry + ' spends'] = sorted_user_spend_spend_list 

## Building Recommendation DataFrame

In [76]:
recommend_user_df = pd.DataFrame(data=industry_user_spend)

## Club Lloyds

In [97]:
# Getting the High Receivers and Creating Dictionary
monthly_spend_list_estimate = [x / 12 for x in user_df['received_tx_sum_gbp']]
club_lloyds_ad_dict = dict(zip(user_df['user_id'], monthly_spend_list_estimate))
sorted_club_lloyds_ad_dict = dict(sorted(club_lloyds_ad_dict.items(), key=lambda item: item[1], reverse=True)[:top_n])

# Adding this to the dataframe
recommend_user_df['Club Lloyds Target users'] = sorted_club_lloyds_ad_dict.keys()
recommend_user_df['Club Lloyds Target monthly recieves'] = sorted_club_lloyds_ad_dict.values()

## Saving Dataframe to CSV

In [99]:
recommend_user_df.to_csv('Results/recommendations.csv', index=False)